In [1]:
#使用Selector提取数据
# Selector对象
# 从页面中提取数据的核心技术是HTTP文本解析，在Python中常用以下模块处理此类问题：
# ●　BeautifulSoup
# BeautifulSoup是非常流行的HTTP解析库，API简洁易用，但解析速度较慢。
# ●　lxml
# lxml是一套由C语言编写的xml解析库（libxml2），解析速度更快，API相对复杂。

# Scrapy综合上述两者优点实现了Selector类，它是基于lxml库构建的，并简化了API接口。在Scrapy中使用Selector对象提取页面中的数据，使用时先
# 通过XPath或CSS选择器选中页面中要提取的数据，然后进行提取。


In [8]:
import scrapy
from scrapy.selector import Selector
# 创建对象
# Selector类的实现位于scrapy.selector模块，创建Selector对象时，可将页面的HTML文档字符串传递给Selector构造器方法的text参数：
text = '''
      <html>
       <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python</b>
            <ul>
            <li>C++</li>
            <li>Java</li>
             <li>Python</li>
            </ul>
            </body>
          </html> '''
selector = Selector(text=text)
print(selector)

<Selector xpath=None data='<html>\n       <body>\n            <h1>...'>


In [20]:
#也可以使用一个Response对象构造Selector对象，将其传递给Selector构造器方法的response参数：
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
body = '''<html>
       <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python</b>
            <ul>
            <li>C++</li>
            <li>Java</li>
             <li>Python</li>
            </ul>
            </body>
          </html> '''
response = HtmlResponse(url='http://www.example.com', body=body,encoding = 'utf-8')
selector = Selector(response=response)
print(selector)
print()

# 选中数据
# 调用Selector对象的xpath方法或css方法（传入XPath或CSS选择器表达式），可以选中文档中的某个或某些部分

selector_list = selector.xpath('//h1')
print(selector_list)
print()

# xpath和css方法返回一个SelectorList对象，其中包含每个被选中部分对应的Selector对象，SelectorList支持列表接口，可使用for语句
# 迭代访问其中的每一个Selector对象
for sel in selector_list:
    print(sel.xpath('./text()'))
    
print()
# SelectorList对象也有xpath和css方法，调用它们的行为是：以接收到的参数分别调用其中每一个Selector对象的xpath或css方法，并将所有结果
# 收集到一个新的SelectorList对象返回给用户
print(selector_list.xpath('./text()'))
print()
print(selector.xpath('.//ul').css('li').xpath('./text()'))
print()

# 提取数据
# 调用Selector或SelectorLis对象的以下方法可将选中的内容提取：
# ●　extract()
# ●　re()
# ●　extract_first() (SelectorList专有)
# ●　re_first() (SelectorList专有)
# 首先来看extract方法，调用Selector对象的extract方法将返回选中内容的Unicode字符串
sl = selector.xpath('.//li')
print(sl)
print()
sl = selector.xpath('.//li/text()')
print(sl)
print()
print(sl[1].extract())
print()


# 与SelectorList对象的xpath和css方法类似，SelectorList对象的extract方法内部会调用其中每个Selector对象的extract方法，并把所
# 有结果收集到一个列表返回给用户
sl = selector.xpath('.//li/text()')
print(sl)
print()
print(sl.extract())
print()


# SelectorList对象还有一个extract_first方法，该方法返回其中第一个Selector对象调用extract方法的结果。通常，在SelectorList对象
# 中只包含一个Selector对象时调用该方法，直接提取出Unicode字符串而不是列表：
sl = selector.xpath('.//b')
print(sl)
print()
print(sl.extract())
print()
print(sl.extract_first())

<Selector xpath=None data='<html>\n       <body>\n            <h1>...'>

[<Selector xpath='//h1' data='<h1>Hello World</h1>'>, <Selector xpath='//h1' data='<h1>Hello Scrapy</h1>'>]

[<Selector xpath='./text()' data='Hello World'>]
[<Selector xpath='./text()' data='Hello Scrapy'>]

[<Selector xpath='./text()' data='Hello World'>, <Selector xpath='./text()' data='Hello Scrapy'>]

[<Selector xpath='./text()' data='C++'>, <Selector xpath='./text()' data='Java'>, <Selector xpath='./text()' data='Python'>]

[<Selector xpath='.//li' data='<li>C++</li>'>, <Selector xpath='.//li' data='<li>Java</li>'>, <Selector xpath='.//li' data='<li>Python</li>'>]

[<Selector xpath='.//li/text()' data='C++'>, <Selector xpath='.//li/text()' data='Java'>, <Selector xpath='.//li/text()' data='Python'>]

Java

[<Selector xpath='.//li/text()' data='C++'>, <Selector xpath='.//li/text()' data='Java'>, <Selector xpath='.//li/text()' data='Python'>]

['C++', 'Java', 'Python']

[<Selector xpath='.//b' data='<b>Hello p

In [26]:

#有些时候，我们想使用正则表达式提取选中内容中的某部分，可以使用re方法（两个对象都有该方法）：
text = '''
    <ul>
    <li>Python 学习手册 <b>价格: 99.00 元</b></li>
    <li>Python 核心编程 <b>价格: 88.00 元</b></li>
     <li>Python 基础教程 <b>价格: 80.00 元</b></li>
    </ul>'''
selector = Selector(text=text)
print(selector.xpath('.//li/b/text()'))
print()
print(selector.xpath('.//li/b/text()').extract())
print()
print(selector.xpath('.//li/b/text()').re('\d+\.\d+')) #只提取价格的数字部分
print()
      
#SelectorList对象的re_first方法同样返回其中的第一个Selector对象调用re方法的结果:
print(selector.xpath('.//li/b/text()').re_first('\d+\.\d+'))

[<Selector xpath='.//li/b/text()' data='价格: 99.00 元'>, <Selector xpath='.//li/b/text()' data='价格: 88.00 元'>, <Selector xpath='.//li/b/text()' data='价格: 80.00 元'>]

['价格: 99.00 元', '价格: 88.00 元', '价格: 80.00 元']

['99.00', '88.00', '80.00']

99.00


In [28]:
#Response内置Selector
from scrapy.http import HtmlResponse
body = '''<html>
       <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python</b>
            <ul>
            <li>C++</li>
            <li>Java</li>
             <li>Python</li>
            </ul>
            </body>
          </html> '''
response = HtmlResponse(url='http://www.example.com', body=body, encoding = 'utf-8')
print(response.selector)
print(response.xpath('.//h1/text()').extract())
print(response.css('li::text').extract())

<Selector xpath=None data='<html>\n       <body>\n            <h1>...'>
['Hello World', 'Hello Scrapy']
['C++', 'Java', 'Python']


In [29]:
# XPath
# XPath即XML路径语言（XML Path Language），它是一种用来确定xml文档中某部分位置的语言
# xml文档的节点有多种类型，其中最常用的有以下几种：
# ●　根节点　整个文档树的根。
# ●　元素节点　html、body、div、p、a。
# ●　属性节点　href。
# ●　文本节点　Hello world、Click here。

# 节点间的关系有以下几种：
# ●　父子　body是html的子节点，p和a是div的子节点。反过来，div是p和a的父节点。
# ●　兄弟　p和a为兄弟节点。
# ●　祖先／后裔　body、div、p、a都是html的后裔节点；反过来html是body、div、p、a的祖先节点。

# <html>
# <body>
# <div >
# <p>Hello world<p>
# <a href="/home">Click here</a>
# </div>
# </body>
# </html>

#基础语法
![title](img/1.png)

In [44]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
body = '''
    <html>
    <head>
    <base href='http://example.com/' />
     <title>Example website</title>
    </head>
    <body>
    <div id='images'>
    <a href='image1.html'>Name: Image 1 <br/><img src='image1.jpg'>
    <a href='image2.html'>Name: Image 2 <br/><img src='image2.jpg'>
    <a href='image3.html'>Name: Image 3 <br/><img src='image3.jpg'>
    <a href='image4.html'>Name: Image 4 <br/><img src='image4.jpg'>
    <a href='image5.html'>Name: Image 5 <br/><img src='image5.jpg'> </div>
    </body>
    </html>'''
response = HtmlResponse(url='http://www.example.com', body=body, encoding='utf-8')
#●　/：描述一个从根开始的绝对路径
print('/：描述一个从根开始的绝对路径',response.xpath('/html'))
print()
#●　E1/E2：选中E1子节点中的所有E2
# 选中div子节点中的所有a
print('选中div子节点中的所有a',response.xpath('/html/body/div/a'))
print()
# 选中body后代中的所有img
print(response.xpath('/html/body//img'))
print()

#●　E/text()：选中E的文本子节点
# 选中所有a的文本
sel = response.xpath('//a/text()')
print(sel)
print(sel.extract())
print()

# 选中html的所有元素子节点
print(response.xpath('/html/*'))
print()

#● E/*：选中E的所有元素子节点
# 选中div的所有后代元素节点
print(response.xpath('/html/body/div//*'))
print()

#● */E：选中孙节点中
# 选中div孙节点中的所有img
print(response.xpath('//div/*/img'))
print()

#● E/@ATTR：选中E的ATTR属性
# 选中所有img的src 属性
print(response.xpath('//img/@src'))

#● //@ATTR：选中文档中所有ATTR属性
# 选中所有的href 属性
print(response.xpath('//@href'))

#● .：选中当前节点，用来描述相对路径
# 获取第1个a的选择器对象
sel = response.xpath('//a')[0]
print(sel)

/：描述一个从根开始的绝对路径 [<Selector xpath='/html' data='<html>\n    <head>\n    <base href="htt...'>]

选中div子节点中的所有a [<Selector xpath='/html/body/div/a' data='<a href="image1.html">Name: Image 1 <...'>, <Selector xpath='/html/body/div/a' data='<a href="image2.html">Name: Image 2 <...'>, <Selector xpath='/html/body/div/a' data='<a href="image3.html">Name: Image 3 <...'>, <Selector xpath='/html/body/div/a' data='<a href="image4.html">Name: Image 4 <...'>, <Selector xpath='/html/body/div/a' data='<a href="image5.html">Name: Image 5 <...'>]

[<Selector xpath='/html/body//img' data='<img src="image1.jpg">'>, <Selector xpath='/html/body//img' data='<img src="image2.jpg">'>, <Selector xpath='/html/body//img' data='<img src="image3.jpg">'>, <Selector xpath='/html/body//img' data='<img src="image4.jpg">'>, <Selector xpath='/html/body//img' data='<img src="image5.jpg">'>]

[<Selector xpath='//a/text()' data='Name: Image 1 '>, <Selector xpath='//a/text()' data='\n    '>, <Selector xpath='//a/text()' data=

In [45]:
#● node[谓语]：谓语用来查找某个特定的节点或者包含某个特定值的节点
# 选中所有a 中的第3 个
print(response.xpath('//a[3]'))

[<Selector xpath='//a[3]' data='<a href="image3.html">Name: Image 3 <...'>]


In [46]:
#●..：选中当前节点的父节点，用来描述相对路径
# 选中所有img的父节点
print(response.xpath('//img/..'))

[<Selector xpath='//img/..' data='<a href="image1.html">Name: Image 1 <...'>, <Selector xpath='//img/..' data='<a href="image2.html">Name: Image 2 <...'>, <Selector xpath='//img/..' data='<a href="image3.html">Name: Image 3 <...'>, <Selector xpath='//img/..' data='<a href="image4.html">Name: Image 4 <...'>, <Selector xpath='//img/..' data='<a href="image5.html">Name: Image 5 <...'>]


In [48]:
#常用函数
# 使用last函数，选中最后1 个
print(response.xpath('//a[last()]'))

[<Selector xpath='//a[last()]' data='<a href="image5.html">Name: Image 5 <...'>]


In [49]:
# 使用position函数，选中前3 个
print(response.xpath('//a[position()<=3]'))

[<Selector xpath='//a[position()<=3]' data='<a href="image1.html">Name: Image 1 <...'>, <Selector xpath='//a[position()<=3]' data='<a href="image2.html">Name: Image 2 <...'>, <Selector xpath='//a[position()<=3]' data='<a href="image3.html">Name: Image 3 <...'>]


In [50]:
# 选中所有含有id属性的div
print(response.xpath('//div[@id]'))

[<Selector xpath='//div[@id]' data='<div id="images">\n    <a href="image1...'>]


In [51]:
# 选中所有含有id属性且值为"images"的div
print(response.xpath('//div[@id="images"]'))

[<Selector xpath='//div[@id="images"]' data='<div id="images">\n    <a href="image1...'>]


In [53]:
#●　string(arg)：返回参数的字符串值
from scrapy.selector import Selector
text='<a href="#">Click here to go to the <strong>Next Page</strong></a>'
sel = Selector(text=text)
print(sel)
# 以下做法和sel.xpath('/html/body/a/strong/text()')得到相同结果
print(sel.xpath('string(/html/body/a/strong)').extract())

# 如果想得到a 中的整个字符串'Click here to go to the Next Page'，
# 使用text()就不行了，因为Click here to go to the和Next Page 在不同元素下
# 以下做法将得到两个子串
print(sel.xpath('/html/body/a//text()').extract())
# 这种情况下可以使用string()函数
print(sel.xpath('string(/html/body/a)').extract())

<Selector xpath=None data='<html><body><a href="#">Click here to...'>
['Next Page']
['Click here to go to the ', 'Next Page']
['Click here to go to the Next Page']


In [ ]:
#contains(str1, str2)：判断str1中是否包含str2，返回布林值
text = '''
    <div>
    <p class="small info">hello world</p>
    <p class="normal info">hello scrapy</p>
     </div>'''
sel = Selector(text=text)
pritn(sel                                                                                                                                                                                                                                                                                                                                                                                                                                                                     )
# 选择class 属性中包含"small"[<Selector xpath='//p[contains(@class, "small")]' data='<p class="small" >
print(sel.xpath('//p[contains(@class, "small")]')) 
# 选择class 属性中包含"info"[<Selector xpath='//p[contains(@class, "info")]' data='<p class="small" >
print(sel.xpath('//p[contains(@class, "info")]'))
